# Artificial intelligence for Ionospheric perturbations DEtections (AIDE)
AIDE allows you to scan vTEC records to detect co-seismic ionospheric perturbations. AIDE software consists of a Machine-Learning based detector, arrival-time picker, and an associator across GNSS satellite networks.

Reference paper: Brissaud, Q., & Astafyeva, E. (2021). Near-real-time detection of co-seismic ionospheric disturbances using machine learning. Geophysical Journal International. doi: 10.1093/gji/ggac167

<center><img src="scheme_ML_improved.png" width="600"/></center>
Figure: AIDE workflow

In [ ]:
%matplotlib inline
import numpy as np
from pdb import set_trace as bp
import pandas as pd
import seaborn as sns

import AIDE, detector, compute_params_waveform, read_data, train_est, train_wave_picker, localization, utils_paper, associator, constants

## Directories and data/model loading
First, select where vTEC data are located and where figures should be generated

In [ ]:
## Directories
main_dir = '/staff/quentin/Documents/Projects/ML_TEC/'
dir_dict = {
    'DIR_FIGURES': main_dir + 'figures/',
    'DIR_DATA': main_dir + 'data/'
}

Then, add entries to `load_dict` in order to load data and/or models instead of re-generating them. Comment line or remove entry to re-generate.

In [ ]:
## Loading previously-generated data/models
load_dict = {
    'est': main_dir + 'figures/model_arr4_noise4_snr1_5.0_pFalse/forest_est_s500.0_m0.7_n1000.0_w720.0.pkl',
    'data': main_dir + 'figures/model_arr4_noise4_snr1_5.0_pFalse/data_s500.0_m0.7_n1000.0_w720.0.pkl',
    'features': main_dir + 'figures/model_arr4_noise4_snr1_5.0_pFalse/features_features_m0.7_w720.0.csv',
    'features-picker': main_dir + 'data/features_picker_w720.0_d0.7.csv',
    'est-picker': main_dir + 'figures/random_est_picker_w720.0_d0.7_bTrue.joblib',
    #'detections': main_dir + 'figures/detected_arrivals_Iquique_afterreview_detrend.csv',
    #'probas': main_dir + 'figures/probas_all_waveforms_Iquique_afterreview_detrend.csv',
    #'associations': main_dir + 'Iquique_afterreview_associations.csv',
    #'associations_time_steps': main_dir + 'Iquique_afterreview_associations_time_steps.csv',
}

## Create an AIDE model instance
To create a new AIDE model instance, you need to: 1) load vTEC data, 2) create an instance of the ML detector, and 3) create an instance of the ML picker

In [ ]:
## Create detection model
one_model = AIDE.AIDE(dir_dict)
one_model.load_data(load_dict)
one_model.load_RF_detector(load_dict)
one_model.load_RF_picker(load_dict)

## Run the detector and arrival-time picker
Run the detector with input parameters defined in `detection_options` over satellite network and events defined in `detection_network`.
- `nb_picks` (int): Number of picks to use to confirm a detections
- `plot_probas` (bool): Whether or not you want to plot the detection probabilities for each input vTEC waveform
- `focus_on_arrival` (bool): Whether or not you want to only perform detections around the true detection time. You need `tec_data_param` entries for this.
- `focus_arrival_shift` (s, float): If `focus_on_arrival=True`, time window used to extract waveforms
- `nb_CPU` (int): Number of CPUs to deploy detections using multiprocessing

In [ ]:
## Perform detection
detection_network = {
    'name': 'test',
    'events': ['Tohoku_1s'],
    'satellites': ['G26'],
    'stations': one_model.tec_data.loc[one_model.tec_data['event'].isin( ['Tohoku_1s'] ) & one_model.tec_data['satellite'].isin( ['G26'] ), 'station'].unique().tolist()[:5],
    'time_end': one_model.tec_data.loc[one_model.tec_data['event'].isin( ['Tohoku_1s'] ), 'time_s'].max()
}

detection_options = { 
    'nb_picks': 5, 
    'plot_probas': True, 
    'focus_on_arrival': False, 
    'focus_arrival_shift': 1000., 
    'nb_CPU': 20
}
one_model.run_detections(load_dict, detection_network, **detection_options)

## Run the associator
Run the associator with the inputs parameters provided in `association_options`.
- `max_radius_search` (m, float): Maximum distance between two ionospheric points for association
- `velocity_search_max` (km/s, float): Maximum horizontal propagation velocity between two ionospheric points for association
- `velocity_search_min` (km/s, float): Minimum horizontal propagation velocity between two ionospheric points for association
- `save_associations` (bool): Whether or not you want to save the `DataFrame` containing associated arrivals
- `association_name` (str): Name for this association dataset

In [ ]:
## Create association list
association_options = {
    'max_radius_search': 500., 
    'velocity_search_max': 50., 
    'velocity_search_min': 0.65, 
    'save_associations': True, 
    'association_name': 'test'
}
one_model.run_association(load_dict, **association_options)

In [ ]:
## Create Figure
first_detections = utils_paper.create_arrival_time_plot(one_model.detections, one_model.options, offset=500., nb_pts_picker=5, quantile_threshold=0.8)
utils_paper.plot_image_iono(one_model.tec_data, one_model.tec_data_param, first_detections, one_model.options, associations=one_model.associations, add_fault=False, add_inset_fault=False, unknown='slip', rotation=25., vmin=6., vmax=11., offset_source_lat=8., offset_source_lon=8., first_label='d', hion_dict={}, add_new_waveform={}, add_new_waveform_class={}, ext_name='_test')